# DC$\pi$BO - Real World Example: Predator-Prey System (ODE)

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../src/")
sys.path.append("../../data/")
sys.path.append("../..")

from src.examples.example_setups import setup_plankton_SEM
from src.utils.sem_utils.real_sems import PredatorPreySEM
from src.utils.sem_utils.sem_estimate import build_sem_hat

from src.experimental.experiments import create_plankton_dataset

from src.experimental.experiments import run_methods_replicates
from src.experimental.analyse_results import get_relevant_results, elaborate, gap_metric_standard, get_common_initial_values, get_converge_trial
from src.utils.plotting import plot_expected_opt_curve_paper

from src.methods.bo import BO
from src.methods.dcbo import DCBO
from src.methods.pibo import PIBO
from src.methods.dcpibo import DCPIBO

from matplotlib.pyplot import rc
import numpy as np
from scipy import stats
import random

random.seed(42)

## Utility Functions

In [2]:
def power_list(input_list, exponent):
    '''Raises each element of the given input_list to the desired exponent'''
    
    return_list = []
    
    for element in input_list:
        if element >= 0:
            raised_element = element**exponent
        else:
            raised_element = -(abs(element)**exponent)
        return_list.append(raised_element)
        
    return return_list

In [3]:
def compute_statistics(y_function, standard_deviation):
    '''Computes all the necessary lists associated to the Normal Distribution'''
    
    return_variates = []
    return_amps = []
    return_pdfs = []
    
    for mean in y_function:
        return_variates.append(stats.norm.rvs(mean, standard_deviation, 10))
        
        amp = np.linspace(mean-5*standard_deviation, mean+5*standard_deviation, 10)
        return_amps.append(amp)
        
        return_pdfs.append(stats.norm.pdf(amp, mean, standard_deviation))
    
    return return_variates, return_amps, return_pdfs

In [4]:
def normalise_pdfs(pdfs_list):
    '''Normalises the PDFs between 0 and 1'''
    
    return_normalised_pdfs_list = []
    
    for pdf_list in pdfs_list:
        temp_list = []
        
        pdf_min = min(pdf_list)
        pdf_max = max(pdf_list)
        
        for pdf_value in pdf_list:
            temp_list.append(round((pdf_value-pdf_min)/(pdf_max-pdf_min),2))
        
        return_normalised_pdfs_list.append(temp_list)
        
    return return_normalised_pdfs_list

In [5]:
def regret_priors(start_prior):
    '''Adds regret'''
    
    return_raised_priors = []
    
    for i in range(1,N+1):
        temp = []
        gamma = beta/i
        
        for p_list in start_prior:
            temp.append(power_list(p_list, gamma))
            
        return_raised_priors.append(temp)
    
    return return_raised_priors

In [6]:
def predict_optima_regret(iterations, time_steps, regret_priors, normalised_pdfs):
    '''Computes the prediction for each time-step and each iteration, according to the effect of gamma (regret)'''
    
    return_predictions = []
    
    for iteration in range(iterations):
        temp = []
        
        for time_step in range(time_steps):
            if(min(regret_priors[iteration][time_step])+max(regret_priors[iteration][time_step]))<0:
                optimum = min(np.multiply(regret_priors[iteration][time_step], normalised_pdfs[time_step]))
            else:
                optimum = max(np.multiply(regret_priors[iteration][time_step], normalised_pdfs[time_step]))
                              
            temp.append(optimum)
                              
        return_predictions.append(temp)
                              
    return return_predictions

## Constants and such

In [7]:
T = 4
t_axis = np.linspace(1,T,T,dtype=int)
N = 20
std = 0.1
beta = 1

## Optima location

### Good prior

In [8]:
a_param = 0
b_param = 0
y = a_param*t_axis+b_param # the line on which I assume the optima would be

### Bad prior

In [9]:
bad_a_param = 6
bad_y_param = 4
bad_y = bad_a_param*t_axis+bad_y_param

## Setup

### Good prior

In [10]:
all_variates, all_amps, all_pdfs = compute_statistics(y, std)
all_variates_regret = regret_priors(all_variates)

all_pdfs_normalised = normalise_pdfs(all_pdfs)

predicted_optima_regret = predict_optima_regret(N, T, all_variates_regret, all_pdfs_normalised) # this are all the 𝜋s

### Bad prior

In [11]:
bad_all_variates, bad_all_amps, bad_all_pdfs = compute_statistics(bad_y, std)
bad_all_variates_regret = regret_priors(bad_all_variates)

bad_all_pdfs_normalised = normalise_pdfs(bad_all_pdfs)

bad_predicted_optima_regret = predict_optima_regret(N, T, bad_all_variates_regret, bad_all_pdfs_normalised) # this are all the 𝜋s

### Model:

In [12]:
init_sem, sem, dag_view, G, exploration_sets, intervention_domain, true_objective_values, optimal_interventions, all_causal_effects = setup_plankton_SEM(T=T)

 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/sem_utils/real_sems.py:55: RuntimeWarning:invalid value encountered in double_scalars


In [13]:
manipulative_vars = list(intervention_domain.keys())

In [14]:
observation_samples = create_plankton_dataset(1,N+1)

## Models' execution

In [15]:
R = 3

### Good Prior

In [16]:
good_results = run_methods_replicates(G=G, 
                                sem=PredatorPreySEM, 
                                make_sem_estimator=build_sem_hat, 
                                base_target_variable='D',
                                intervention_domain = intervention_domain, 
                                methods_list = ['BO', 'PIBO', 'DCBO', 'DCPIBO'],
                                obs_samples = observation_samples,
                                exploration_sets = exploration_sets,
                                priors_regret = predicted_optima_regret,
                                total_timesteps = T,
                                number_of_trials = N, 
                                reps = R, # Number of replicates (how many times we run each method)
                                n_restart = 1,
                                save_data = False,
                                n_obs = 5, # The method samples 5 time-series for each replicate
                                num_anchor_points = 100,
                                sample_anchor_points = True,
                                controlled_experiment=False,
                                manipulative_variables=manipulative_vars)

Experiment count:   0%|                                                                           | 0/3 [00:00<?, ?it/s]


	>>>BO




Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/sem_utils/real_sems.py:55: RuntimeWarning:invalid value encountered in double_scalars

Time index:  25%|██████████████████▎                                                      | 1/4 [00:00<00:00,  3.97it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  50%|████████████████████████████████████▌                                    | 2/4 [00:00<00:00,  3.74it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [00:00<00:00,  3.74it/s]reconstraining parameters GP_regression.rbf.variance

Time index: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.76it/s]


20

	>>>PIBO




Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  25%|██████████████████▎                                                      | 1/4 [00:00<00:00,  3.79it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  50%|████████████████████████████████████▌                                    | 2/4 [00:00<00:00,  3.73it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [00:00<00:00,  3.68it/s]reconstraining parameters GP_regression.rbf.variance

Time index: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.71it/s]
 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead o

...


Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  25%|██████████████████▎                                                      | 1/4 [00:00<00:00,  3.79it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  50%|████████████████████████████████████▌                                    | 2/4 [00:00<00:00,  3.77it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [00:00<00:00,  3.70it/s]reconstraining parameters GP_regression.rbf.variance

Time index: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.77it/s]



	>>>DCBO



 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply

Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance

Time index:  25%|██████████████████▎                                                      | 1/4 [00:26<01:18, 26.08


	>>>DCPIBO



 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply

Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:199: RuntimeWar

### Bad prior

In [17]:
bad_results = run_methods_replicates(G=G, 
                                sem=PredatorPreySEM, 
                                make_sem_estimator=build_sem_hat, 
                                base_target_variable='D',
                                intervention_domain = intervention_domain, 
                                methods_list = ['BO', 'PIBO', 'DCBO', 'DCPIBO'],
                                obs_samples = observation_samples,
                                exploration_sets = exploration_sets,
                                priors_regret = bad_predicted_optima_regret,
                                total_timesteps = T,
                                number_of_trials = N, 
                                reps = R, # Number of replicates (how many times we run each method)
                                n_restart = 1,
                                save_data = False,
                                n_obs = 5, # The method samples 5 time-series for each replicate
                                num_anchor_points = 100,
                                sample_anchor_points = True,
                                controlled_experiment=False,
                                manipulative_variables=manipulative_vars)

Experiment count:   0%|                                                                           | 0/3 [00:00<?, ?it/s]


	>>>BO




Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  25%|██████████████████▎                                                      | 1/4 [00:00<00:00,  4.04it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  50%|████████████████████████████████████▌                                    | 2/4 [00:00<00:00,  3.79it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [00:00<00:00,  3.77it/s]reconstraining parameters GP_regression.rbf.variance

Time index: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.72it/s]


20

	>>>PIBO




Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  25%|██████████████████▎                                                      | 1/4 [00:00<00:00,  3.79it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  50%|████████████████████████████████████▌                                    | 2/4 [00:00<00:00,  3.67it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [00:00<00:00,  3.65it/s]reconstraining parameters GP_regression.rbf.variance

Time index: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.63it/s]


...


Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  25%|██████████████████▎                                                      | 1/4 [00:00<00:00,  3.85it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  50%|████████████████████████████████████▌                                    | 2/4 [00:00<00:00,  3.78it/s]reconstraining parameters GP_regression.rbf.variance

Time index:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [00:00<00:00,  3.86it/s]reconstraining parameters GP_regression.rbf.variance

Time index: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.81it/s]



	>>>DCBO



 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply

Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:199: RuntimeWar


	>>>DCPIBO



 /mnt/c/Users/lucal/Desktop/Thesis/notebooks/MyTests/../../src/utils/dag_utils/adjacency_matrix_utils.py:26: FutureWarning:adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply

Time index:   0%|                                                                                 | 0/4 [00:00<?, ?it/s]reconstraining parameters GP_regression.rbf.variance
reconstraining parameters GP_regression.rbf.variance
 /home/luca/miniconda3/envs/MTL_DCBO_testing/lib/python3.10/site-packages/GPy/kern/src/stationary.py:199: RuntimeWar

## Plotting

In [18]:
plot_params = {
    "linewidth": 3,
    "linewidth_opt": 4,
    "alpha": 0.1,
    "xlim_max": N,
    "ncols": 5,
    "loc_legend": "lower right",
    "size_ticks": 20,
    "size_labels": 20,
    "xlabel": r'$\texttt{cost}(\mathbf{X}_{s,t}, \mathbf{x}_{s,t})$',
    "labels": {'DCPIBO': 'DCPIBO', 'PIBO': 'PIBO', 'DCBO': 'DCBO', 'BO': 'BO', 'True': r'$\mathbb{E} \left [Y_t \mid \textrm{do}(\mathbf{X}_{s,t}^\star = \mathbf{x}_{s,t}^\star) \right]$'},
    "colors": {'DCPIBO': 'blue', 'PIBO': 'green', 'DCBO': 'orange', 'BO': 'red', 'True': 'black'},
    "line_styles": {'DCPIBO': 'dashdot', 'PIBO': '--', 'DCBO': '-', 'BO': '-', 'True': ':'},
    "width":10
}
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{amssymb}')
rc('font', family='serif')
rc('font', size=20)

### Good prior

In [19]:
good_data = get_relevant_results(results=good_results, replicates=R)
good_exp_optimal_outcome_values_during_trials, good_exp_per_trial_cost = elaborate(number_of_interventions=None, 
                                                                         n_replicates=R, 
                                                                         data=good_data, 
                                                                         best_objective_values=true_objective_values, 
                                                                         T=T)

In [20]:
# Each frame corresponds to one time-slice.
plot_expected_opt_curve_paper(T,
    true_objective_values,
    good_exp_per_trial_cost,
    good_exp_optimal_outcome_values_during_trials,
    plot_params,  
    fig_size = (15,5))

### Bad prior

In [21]:
bad_data = get_relevant_results(results=bad_results,replicates=R)
bad_exp_optimal_outcome_values_during_trials, bad_exp_per_trial_cost = elaborate(number_of_interventions=None, 
                                                                         n_replicates=R, 
                                                                         data=bad_data, 
                                                                         best_objective_values=true_objective_values, 
                                                                         T=T)

In [22]:
# Each frame corresponds to one time-slice.
plot_expected_opt_curve_paper(T,
    true_objective_values,
    bad_exp_per_trial_cost,
    bad_exp_optimal_outcome_values_during_trials,
    plot_params,  
    fig_size = (15,5))

## Gap Metric

In [30]:
def compute_gap_values(T, summary):
    G_BO=0
    G_PIBO=0
    G_DCBO=0
    G_DCPIBO=0
    list_gaps=[]
    summary_list = list(summary.values())
    for i in range(len(summary_list)):
        for t in range(T):
            # print(summary_list[i][t][0])
            if i==0:
                G_BO=G_BO+summary_list[i][t][0]
                if t==3:
                    list_gaps.append(G_BO/T)
            elif i==1:
                G_PIBO=G_PIBO+summary_list[i][t][0]
                if t==3:
                    list_gaps.append(G_PIBO/T)
            elif i==2:
                G_DCBO=G_DCBO+summary_list[i][t][0]
                if t==3:
                    list_gaps.append(G_DCBO/T)
            elif i==3:
                G_DCPIBO=G_DCPIBO+summary_list[i][t][0]
                if t==3:
                    list_gaps.append(G_DCPIBO/T)
    return list_gaps

### Good Gap Metric

In [31]:
good_t_i_l = get_common_initial_values(T=T,
                                  data=good_data,
                                  n_replicates=R)

good_w_c_d = get_converge_trial(best_objective_values=true_objective_values,
                           exp_optimal_outcome_values_during_trials=good_exp_optimal_outcome_values_during_trials,
                           n_trials=N,
                           T=T,
                           n_decimal=1)

good_summary = gap_metric_standard(T=T,
                             data=good_data,
                             best_objective_values=true_objective_values,
                             total_initial_list = good_t_i_l,
                             n_replicates=R,
                             n_trials=N,
                             where_converge_dict=good_w_c_d)

good_list = compute_gap_values(T, good_summary)

In [32]:
good_summary

{'BO': [[0.4522550915258394, 0.022862242254376355],
  [0.48197847355794515, 0.02973439130105232],
  [0.49072795642718364, 0.02830267462841793],
  [0.45586501968339216, 0.016676684133411838]],
 'PIBO': [[0.41299566817110417, 0.0134734935590289],
  [0.5384615384615385, 0.0],
  [0.4233944940155099, 0.025344256024698208],
  [0.6849808053770499, 0.005514636877260893]],
 'DCBO': [[0.5016410352533204, 0.0013157799027521328],
  [0.48783394104022726, 0.035336348688888755],
  [0.8410644005766669, 0.0036697419062039632],
  [0.5038246283629678, 0.012722101805401592]],
 'DCPIBO': [[0.4979360770581676, 0.0036704329414789906],
  [0.5384615384615385, 0.0],
  [0.5031537787465703, 0.004163005782827279],
  [0.8974358974358975, 0.0]]}

In [33]:
good_list

[0.4702066352985901,
 0.5149581265063006,
 0.5835910013082956,
 0.6092468229255434]

### Bad Gap Metric

In [34]:
bad_t_i_l = get_common_initial_values(T=T,
                                  data=bad_data,
                                  n_replicates=R)

bad_w_c_d = get_converge_trial(best_objective_values=true_objective_values,
                          exp_optimal_outcome_values_during_trials=bad_exp_optimal_outcome_values_during_trials,
                          n_trials=N,
                          T=T,
                          n_decimal=1)

bad_summary = gap_metric_standard(T=T,
                             data=bad_data,
                             best_objective_values=true_objective_values,
                             total_initial_list = bad_t_i_l,
                             n_replicates=R,
                             n_trials=N,
                             where_converge_dict=bad_w_c_d)

bad_list = compute_gap_values(T, bad_summary)

In [35]:
bad_summary

{'BO': [[0.4669643713847094, 0.013525704276240643],
  [0.38257553348632367, 0.10972547939295707],
  [0.478744894856771, 0.024237110083938064],
  [0.4476051052616696, 0.014704597110870668]],
 'PIBO': [[0.4590697614023118, 0.05321875806836122],
  [0.4331556024373917, 0.034306506238949014],
  [0.4560866210784195, 0.04063326469129515],
  [0.44185460195901166, 0.02615574838493645]],
 'DCBO': [[0.5038477959624049, 0.0010492292508026925],
  [0.49469877375588966, 0.025628009158976457],
  [0.9165079757886846, 0.004848334015118011],
  [0.5009759790773677, 0.016750700259541656]],
 'DCPIBO': [[0.5035556490194545, 0.0007020761541879235],
  [0.5384615384615385, 0.0],
  [0.506721506389144, 0.0004275800289095355],
  [0.9485743121595177, 0.00020313276898583128]]}

In [36]:
bad_list

[0.4439724762473684,
 0.4475416467192837,
 0.6040076311460867,
 0.6243282515074137]